In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime



/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#model using macro
train = pd.read_csv("/Users/xiechenwei/Downloads/train.csv", parse_dates=['timestamp'])
test = pd.read_csv("/Users/xiechenwei/Downloads/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("/Users/xiechenwei/Downloads/macro.csv", parse_dates=['timestamp'])



In [3]:
#clean data
bad_index = train[train.life_sq > train.full_sq].index
train.ix[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]
bad_index = test[test.life_sq > test.full_sq].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.ix[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
test.ix[bad_index, "full_sq"] = np.NaN
kitch_is_build_year = [13117]
train.ix[kitch_is_build_year, "build_year"] = train.ix[kitch_is_build_year, "kitch_sq"]
bad_index = train[train.kitch_sq >= train.life_sq].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[test.kitch_sq >= test.life_sq].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
test.ix[bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > 300].index
train.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test[test.life_sq > 200].index
test.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)
bad_index = train[train.build_year < 1500].index
good_index = train[train.build_year >= 1500].index

# year #
train['year'] = train['timestamp'].dt.year
test['year'] = test['timestamp'].dt.year

train.ix[bad_index, "build_year"] = np.NaN
train['age_build'] = np.NaN
test['age_build'] = np.NaN
train.ix[good_index, "age_build"] = train['year'] - train['build_year']
good_index = test[train.build_year >= 1500].index
test.ix[good_index, "age_build"] = test['year'] - test['build_year']
bad_index = test[test.build_year < 1500].index
test.ix[bad_index, "build_year"] = np.NaN
bad_index = train[train.num_room == 0].index 
train.ix[bad_index, "num_room"] = np.NaN
bad_index = test[test.num_room == 0].index 
test.ix[bad_index, "num_room"] = np.NaN
bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.ix[bad_index, "num_room"] = np.NaN
bad_index = [3174, 7313]
test.ix[bad_index, "num_room"] = np.NaN
bad_index = train[(train.floor == 0).values * (train.max_floor == 0).values].index
train.ix[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train[train.floor == 0].index
train.ix[bad_index, "floor"] = np.NaN
bad_index = train[train.max_floor == 0].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.max_floor == 0].index
test.ix[bad_index, "max_floor"] = np.NaN
bad_index = train[train.floor > train.max_floor].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.floor > test.max_floor].index
test.ix[bad_index, "max_floor"] = np.NaN
train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.ix[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.ix[bad_index, "state"] = np.NaN
test.state.value_counts()

# brings error down a lot by removing extreme price per sqm
train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]

# Add month-year
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek

y_train = train["price_doc"]



/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  app.launch_new_instance()
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [4]:


# Any results you write to the current directory are saved as output.

train.drop(train[train["life_sq"] > 7000].index, inplace=True)

#I don't know why doing this ???
mult = 0.986
y_train = train['price_doc'].values  * mult + 10

id_test = test['id']

train.drop(['id', 'price_doc'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

num_train = len(train)
df_all = pd.concat([train, test])
# Next line just adds a lot of NA columns (becuase "join" only works on indexes)
# but somewhow it seems to affect the result
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)

df_all.head()

factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

df_columns = df_values.columns





(38058, 396)
(38058, 396)
(38058, 396)


In [5]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}



In [6]:
dtrain = xgb.DMatrix(X_train, y_train, feature_names=df_columns)
dtest = xgb.DMatrix(X_test, feature_names=df_columns)




In [7]:
cv_test = xgb.cv(xgb_params, dtrain, num_boost_round=600,nfold = 4, seed = 0, stratified = True, early_stopping_rounds=50,
               verbose_eval = 1, show_stdv=True)

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=4.
  % (min_labels, self.n_folds)), Warning)


[0]	train-rmse:8.07317e+06+8943.69	test-rmse:8.07935e+06+31307.4
[1]	train-rmse:7.72373e+06+8597.13	test-rmse:7.73488e+06+35192
[2]	train-rmse:7.39615e+06+9987.62	test-rmse:7.41195e+06+40872.5
[3]	train-rmse:7.08754e+06+4717.31	test-rmse:7.10584e+06+46852.2
[4]	train-rmse:6.79877e+06+2925.31	test-rmse:6.82074e+06+50590.9
[5]	train-rmse:6.51657e+06+2250.98	test-rmse:6.54346e+06+51519.3
[6]	train-rmse:6.25496e+06+8300.95	test-rmse:6.28682e+06+50184.2
[7]	train-rmse:6.01042e+06+9305.87	test-rmse:6.04682e+06+54204.5
[8]	train-rmse:5.77712e+06+10785.9	test-rmse:5.81764e+06+52891.3
[9]	train-rmse:5.55925e+06+15267.3	test-rmse:5.60615e+06+47271.9
[10]	train-rmse:5.35372e+06+18675.8	test-rmse:5.40723e+06+51650.9
[11]	train-rmse:5.15803e+06+10888.8	test-rmse:5.22013e+06+56734.6
[12]	train-rmse:4.9765e+06+15565.3	test-rmse:5.04567e+06+57929.3
[13]	train-rmse:4.8e+06+16638.2	test-rmse:4.8746e+06+58885.8
[14]	train-rmse:4.63683e+06+19985.8	test-rmse:4.71834e+06+62334.2
[15]	train-rmse:4.48218e+06+

In [8]:
num_boost_round = 489  # From Bruno's original CV, I think
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_round)

y_pred = model.predict(dtest)

#df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

#df_sub.head()
#first_result = output.merge(df_sub, on="id", suffixes=['_louis','_bruno'])
#first_result["price_doc"] = np.exp( .714*np.log(first_result.price_doc_louis) + 
#                                    .286*np.log(first_result.price_doc_bruno) )  # multiplies out to .5 & .2
#result = first_result.merge(gunja_output, on="id", suffixes=['_follow','_gunja'])
#
#result["price_doc"] = np.exp( .7*np.log(result.price_doc_follow) + 
#                              .3*np.log(result.price_doc_gunja) )
#result.drop(["price_doc_louis","price_doc_bruno","price_doc_follow","price_doc_gunja"],axis=1,inplace=True)
#result.head()
#result.to_csv('/Users/xiechenwei/Downloads/sub2.csv', index=False)

In [9]:
macro_xgb_naive_output = pd.DataFrame({'id': id_test, 'price_doc': y_pred})
macro_xgb_naive_output.head()

,id,price_doc
0,30474,5641154.5
1,30475,8216323.0
2,30476,5378318.0
3,30477,6429196.5
4,30478,5278451.5


In [10]:
macro_xgb_naive_output.to_csv('/Users/xiechenwei/Downloads/macro_xgb_naive_output.csv', index=False)